In [1]:
#The goal is to Start a sushi-restaurant in the city of Brussels
#First we'll look for all current sushi-restaurants and decide on possible locations where there are not many sushi restaurants around
    #For this step we'll use foursqaure to search for specific venue's
    #The information will be processed into a dataframe and plotted onto a map

#We'll then check for the rating of the restaurants near optimal locations
    #For this step we'll use foursquare again to list the ratings of specific restaurant
    #The restaurants will be grouped into clusters based on their rating

#Finally a location can be chosen based on the final map

In [2]:
#Importing and installing necessary libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
#Adding Foursquare information to variables:
CLIENT_ID = 'IL2QNRRCV02PKZRV1WPPAEXVML4XABBERFDZOTCMY4FWRWAY' # your Foursquare ID
CLIENT_SECRET = 'LXGUBXTXOXZYXGBFI5MT03LUP5PTUUHS0C21YM353VW3SKYW' # your Foursquare Secret
VERSION = '20190925'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: IL2QNRRCV02PKZRV1WPPAEXVML4XABBERFDZOTCMY4FWRWAY


In [4]:
link = "https://en.wikipedia.org/wiki/Provinces_of_the_Netherlands"

tables = pd.read_html(link)

Provinces = tables[2].iloc[0:]

Provinces.columns = ['Province','arms', 'capital','largest muni','king com','km1','mi1','km2','mi2','Population','density','density per km2', 'GRP1', 'GRP2']

Provinces.head()

,Province,arms,capital,largest muni,king com,km1,mi1,km2,mi2,Population,density,density per km2,GRP1,GRP2
0,Drenthe,NaN,Assen,Emmen,Jetta Klijnsma,2680,1035,2641,1020,492100,184/km2 (480/sq mi),186/km2 (480/sq mi),14119,28802
1,Flevoland,NaN,Lelystad,Almere,Leen Verbeek,2412,931,1418,547,411670,171/km2 (440/sq mi),290/km2 (750/sq mi),12959,31923
2,Friesland[B],NaN,Leeuwarden,Leeuwarden,Arno Brok,5749,2220,3342,1290,647268,113/km2 (290/sq mi),194/km2 (500/sq mi),18581,28743
3,Gelderland,NaN,Arnhem,Nijmegen,John Berends,5136,1983,4972,1920,2060103,"401/km2 (1,040/sq mi)","414/km2 (1,070/sq mi)",70789,34673
4,Groningen[C],NaN,Groningen,Groningen,René Paas,2960,1143,2333,901,582944,197/km2 (510/sq mi),250/km2 (650/sq mi),24102,41295


In [5]:
Provinces.drop(['arms', 'capital','largest muni','king com','km1','mi1','km2','mi2','density', 'GRP1', 'GRP2'], 1, inplace=True)
Provinces.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Province,Population,density per km2
0,Drenthe,492100,186/km2 (480/sq mi)
1,Flevoland,411670,290/km2 (750/sq mi)
2,Friesland[B],647268,194/km2 (500/sq mi)
3,Gelderland,2060103,"414/km2 (1,070/sq mi)"
4,Groningen[C],582944,250/km2 (650/sq mi)


In [6]:
Province_list = Provinces['Province'].tolist()
Province_list

['Drenthe',
 'Flevoland',
 'Friesland[B]',
 'Gelderland',
 'Groningen[C]',
 'Limburg',
 'North Brabant',
 'North Holland',
 'Overijssel',
 'South Holland',
 'Utrecht',
 'Zeeland']

In [7]:
Province_list = ['Drenthe',
 'Flevoland',
 'Friesland',
 'Gelderland',
 'Groningen',
 'Limburg',
 'North Brabant',
 'North Holland',
 'Overijssel',
 'South Holland',
 'Utrecht',
 'Zeeland']

In [8]:
Provinces['Province'] = Province_list
Provinces.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Province,Population,density per km2
0,Drenthe,492100,186/km2 (480/sq mi)
1,Flevoland,411670,290/km2 (750/sq mi)
2,Friesland,647268,194/km2 (500/sq mi)
3,Gelderland,2060103,"414/km2 (1,070/sq mi)"
4,Groningen,582944,250/km2 (650/sq mi)


In [9]:
House_price = [251000, 275000, 227000, 320000, 228000, 251000, 332000, 431000, 262000, 367000, 454000, 238000]
Price = pd.DataFrame(House_price)
Price.head()

,0
0,251000
1,275000
2,227000
3,320000
4,228000


In [10]:
Data = Provinces
Data['price'] = Price[0]
Data.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Province,Population,density per km2,price
0,Drenthe,492100,186/km2 (480/sq mi),251000
1,Flevoland,411670,290/km2 (750/sq mi),275000
2,Friesland,647268,194/km2 (500/sq mi),227000
3,Gelderland,2060103,"414/km2 (1,070/sq mi)",320000
4,Groningen,582944,250/km2 (650/sq mi),228000


In [11]:
density = Data['density per km2'].tolist()
density

['186/km2 (480/sq\xa0mi)',
 '290/km2 (750/sq\xa0mi)',
 '194/km2 (500/sq\xa0mi)',
 '414/km2 (1,070/sq\xa0mi)',
 '250/km2 (650/sq\xa0mi)',
 '519/km2 (1,340/sq\xa0mi)',
 '514/km2 (1,330/sq\xa0mi)',
 '1,060/km2 (2,700/sq\xa0mi)',
 '346/km2 (900/sq\xa0mi)',
 '1,307/km2 (3,390/sq\xa0mi)',
 '935/km2 (2,420/sq\xa0mi)',
 '214/km2 (550/sq\xa0mi)']

In [12]:
density = [186,
 290,
 194,
 414,
 250,
 519,
 514,
 1060,
 346,
 1307,
 935,
 214]

In [13]:
df_density = pd.DataFrame(density)

In [14]:
Data['density per km2'] = df_density[0]
Data

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Province,Population,density per km2,price
0,Drenthe,492100,186,251000
1,Flevoland,411670,290,275000
2,Friesland,647268,194,227000
3,Gelderland,2060103,414,320000
4,Groningen,582944,250,228000
5,Limburg,1117198,519,251000
6,North Brabant,2528286,514,332000
7,North Holland,2831182,1060,431000
8,Overijssel,1151501,346,262000
9,South Holland,3681044,1307,367000


In [15]:
Provinces_correct = Data['Province'].tolist()
Provinces_correct

['Drenthe',
 'Flevoland',
 'Friesland',
 'Gelderland',
 'Groningen',
 'Limburg',
 'North Brabant',
 'North Holland',
 'Overijssel',
 'South Holland',
 'Utrecht',
 'Zeeland']

In [16]:
Provinces_correct = ['Drenthe',
 'Flevoland',
 'Friesland (Fryslân)',
 'Gelderland',
 'Groningen',
 'Limburg',
 'Noord-Brabant',
 'Noord-Holland',
 'Overijssel',
 'Zuid-Holland',
 'Utrecht',
 'Zeeland']
Provinces_correct_df = pd.DataFrame(Provinces_correct)

In [17]:
Data['Province'] = Provinces_correct
Data

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Province,Population,density per km2,price
0,Drenthe,492100,186,251000
1,Flevoland,411670,290,275000
2,Friesland (Fryslân),647268,194,227000
3,Gelderland,2060103,414,320000
4,Groningen,582944,250,228000
5,Limburg,1117198,519,251000
6,Noord-Brabant,2528286,514,332000
7,Noord-Holland,2831182,1060,431000
8,Overijssel,1151501,346,262000
9,Zuid-Holland,3681044,1307,367000


In [18]:
!wget --quiet https://www.webuildinternet.com/articles/2015-07-19-geojson-data-of-the-netherlands/provinces.geojson -O provinces.geojson
    
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


In [19]:
Netherlands_geo = r'provinces.geojson' # geojson file

Netherlands_density_map = folium.Map(location=[52.370216, 4.895168], zoom_start=7, tiles='Mapbox Bright')

In [20]:
# generate choropleth map
Netherlands_density_map.choropleth(
    geo_data=Netherlands_geo,
    data=Data,
    columns=['Province', 'density per km2'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Density per km2 per Province',
    threshold_scale=[100, 350, 600, 850, 1100, 1350]
)

# display map
Netherlands_density_map

In [21]:
Netherlands_price_map = folium.Map(location=[52.370216, 4.895168], zoom_start=7, tiles='Mapbox Bright')
# generate choropleth map
Netherlands_price_map.choropleth(
    geo_data=Netherlands_geo,
    data=Data,
    columns=['Province', 'price'],
    key_on='feature.properties.name',
    fill_color='Blues', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Average price per housing',
    threshold_scale=[200000, 275000, 350000, 425000, 500000]
)

# display map
Netherlands_price_map

In [22]:
Data['Price per Density'] = Data['price']/Data['density per km2']
Data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Province,Population,density per km2,price,Price per Density
0,Drenthe,492100,186,251000,1349.462366
1,Flevoland,411670,290,275000,948.275862
2,Friesland (Fryslân),647268,194,227000,1170.103093
3,Gelderland,2060103,414,320000,772.946860
4,Groningen,582944,250,228000,912.000000


In [23]:
Data['Price per Density'] = Data['Price per Density'].astype(float).round(1)
Data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Province,Population,density per km2,price,Price per Density
0,Drenthe,492100,186,251000,1349.5
1,Flevoland,411670,290,275000,948.3
2,Friesland (Fryslân),647268,194,227000,1170.1
3,Gelderland,2060103,414,320000,772.9
4,Groningen,582944,250,228000,912.0


In [24]:
Netherlands_prds_map = folium.Map(location=[52.370216, 4.895168], zoom_start=7, tiles='Mapbox Bright')
# generate choropleth map
Netherlands_prds_map.choropleth(
    geo_data=Netherlands_geo,
    data=Data,
    columns=['Province', 'Price per Density'],
    key_on='feature.properties.name',
    fill_color='RdPu', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='House pricing to density ratio',
    threshold_scale=[200, 300, 400, 600, 800, 1400]
)

# display map
Netherlands_prds_map

In [25]:
#Den Haag
# create map of New York using latitude and longitude values
latitude = 52.0704978
longitude = 4.3006999

Denhaag_map = folium.Map(location=[latitude, longitude], zoom_start=12)

Denhaag_map

In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
search_query = 'sushi'
radius = 10000 # define radius


url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,54cfb8ca498e1f08d7d9e639,Rabbijn Maarsenplein 1,NL,Den Haag,Nederland,NaN,1132,"[Rabbijn Maarsenplein 1, 2512 HJ Den Haag, Ned...","[{'label': 'display', 'lat': 52.07616, 'lng': ...",52.076160,4.314450,NaN,2512 HJ,Zuid-Holland,Momiji Sushi,v-1569756233,NaN
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4aaa969bf964a5207a5620e3,Prinsestraat 77,NL,Den Haag,Nederland,NaN,1053,"[Prinsestraat 77, 2513 CC Den Haag, Nederland]","[{'label': 'display', 'lat': 52.07967916645264...",52.079679,4.304444,NaN,2513 CC,Zuid-Holland,Noh Sushi & Ramen,v-1569756233,148083441
2,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,508063d9e4b0d50649b3aee1,Westeinde 199,NL,Den Haag,Nederland,Lange Lombardstraat,487,"[Westeinde 199 (Lange Lombardstraat), 2512GZ D...","[{'label': 'display', 'lat': 52.07475836441009...",52.074758,4.299023,NaN,2512GZ,Zuid-Holland,Sushi & Me,v-1569756233,NaN
3,"[{'id': '5267e4d9e4b0ec79466e48c7', 'name': 'F...",False,5cdff4ccfc9e94002ca78978,NaN,NL,NaN,Nederland,NaN,1606,[Nederland],"[{'label': 'display', 'lat': 52.08274, 'lng': ...",52.082740,4.313126,NaN,NaN,NaN,Sushi & Asian Street Food Festival Joy - Den Haag,v-1569756233,NaN
4,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,5c8256cdb04056002cadccfc,Stationplein 21,NL,Den Haag,Nederland,NaN,1362,"[Stationplein 21, 2515 BT Den Haag, Nederland]","[{'label': 'display', 'lat': 52.069533, 'lng':...",52.069533,4.320553,NaN,2515 BT,Zuid-Holland,Poke Sushi Express,v-1569756233,NaN


In [27]:
dataframe.shape

(48, 19)

In [28]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Momiji Sushi,Sushi Restaurant,Rabbijn Maarsenplein 1,NL,Den Haag,Nederland,NaN,1132,"[Rabbijn Maarsenplein 1, 2512 HJ Den Haag, Ned...","[{'label': 'display', 'lat': 52.07616, 'lng': ...",52.076160,4.314450,NaN,2512 HJ,Zuid-Holland,54cfb8ca498e1f08d7d9e639
1,Noh Sushi & Ramen,Sushi Restaurant,Prinsestraat 77,NL,Den Haag,Nederland,NaN,1053,"[Prinsestraat 77, 2513 CC Den Haag, Nederland]","[{'label': 'display', 'lat': 52.07967916645264...",52.079679,4.304444,NaN,2513 CC,Zuid-Holland,4aaa969bf964a5207a5620e3
2,Sushi & Me,Sushi Restaurant,Westeinde 199,NL,Den Haag,Nederland,Lange Lombardstraat,487,"[Westeinde 199 (Lange Lombardstraat), 2512GZ D...","[{'label': 'display', 'lat': 52.07475836441009...",52.074758,4.299023,NaN,2512GZ,Zuid-Holland,508063d9e4b0d50649b3aee1
3,Sushi & Asian Street Food Festival Joy - Den Haag,Festival,NaN,NL,NaN,Nederland,NaN,1606,[Nederland],"[{'label': 'display', 'lat': 52.08274, 'lng': ...",52.082740,4.313126,NaN,NaN,NaN,5cdff4ccfc9e94002ca78978
4,Poke Sushi Express,Sushi Restaurant,Stationplein 21,NL,Den Haag,Nederland,NaN,1362,"[Stationplein 21, 2515 BT Den Haag, Nederland]","[{'label': 'display', 'lat': 52.069533, 'lng':...",52.069533,4.320553,NaN,2515 BT,Zuid-Holland,5c8256cdb04056002cadccfc
5,MAMBOO Sushi,Sushi Restaurant,Raamstraat 15,NL,Den Haag,Nederland,NaN,883,"[Raamstraat 15, 2512BX Den Haag, Nederland]","[{'label': 'display', 'lat': 52.07605214617236...",52.076052,4.309921,NaN,2512BX,Zuid-Holland,592c075c286fda44e172d413
6,SushiPoint Den Haag,Sushi Restaurant,Laan van Meerdervoort 168 B,NL,Den Haag,Nederland,NaN,1519,"[Laan van Meerdervoort 168 B, 2517 BG Den Haag...","[{'label': 'display', 'lat': 52.0817719, 'lng'...",52.081772,4.288170,Zeeheldenkwartier,2517 BG,Zuid-Holland,52308da4498e8f61ae217b7f
7,Sushi Aino,Sushi Restaurant,Laan Van Meerdervoort 322a,NL,Den Haag,Nederland,NaN,1826,"[Laan Van Meerdervoort 322a, Den Haag, Nederland]","[{'label': 'display', 'lat': 52.07887742188163...",52.078877,4.277744,NaN,NaN,Zuid-Holland,525ab1fa498ea60208f9cd62
8,Urban Sushi,Sushi Restaurant,Pletterijkade 15-28,NL,Den Haag,Nederland,NaN,1501,"[Pletterijkade 15-28, 2515 SG Den Haag, Nederl...","[{'label': 'display', 'lat': 52.07438323311686...",52.074383,4.321709,NaN,2515 SG,Zuid-Holland,5112a5bee4b0c57623b89964
9,Sushi den haag centraal,Sushi Restaurant,Den Haag Cs,NL,Den Haag,Nederland,NaN,1619,"[Den Haag Cs, Den Haag, Nederland]","[{'label': 'display', 'lat': 52.070644, 'lng':...",52.070644,4.324361,NaN,NaN,Zuid-Holland,4e131c226284edb6bae5f52c


In [29]:
dataframe_filtered.dropna(subset=['address'], how='all', inplace = True)
dataframe_filtered.reset_index(drop=True, inplace=True)
dataframe_filtered.shape

(36, 16)

In [30]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Momiji Sushi,Sushi Restaurant,Rabbijn Maarsenplein 1,NL,Den Haag,Nederland,NaN,1132,"[Rabbijn Maarsenplein 1, 2512 HJ Den Haag, Ned...","[{'label': 'display', 'lat': 52.07616, 'lng': ...",52.076160,4.314450,NaN,2512 HJ,Zuid-Holland,54cfb8ca498e1f08d7d9e639
1,Noh Sushi & Ramen,Sushi Restaurant,Prinsestraat 77,NL,Den Haag,Nederland,NaN,1053,"[Prinsestraat 77, 2513 CC Den Haag, Nederland]","[{'label': 'display', 'lat': 52.07967916645264...",52.079679,4.304444,NaN,2513 CC,Zuid-Holland,4aaa969bf964a5207a5620e3
2,Sushi & Me,Sushi Restaurant,Westeinde 199,NL,Den Haag,Nederland,Lange Lombardstraat,487,"[Westeinde 199 (Lange Lombardstraat), 2512GZ D...","[{'label': 'display', 'lat': 52.07475836441009...",52.074758,4.299023,NaN,2512GZ,Zuid-Holland,508063d9e4b0d50649b3aee1
3,Poke Sushi Express,Sushi Restaurant,Stationplein 21,NL,Den Haag,Nederland,NaN,1362,"[Stationplein 21, 2515 BT Den Haag, Nederland]","[{'label': 'display', 'lat': 52.069533, 'lng':...",52.069533,4.320553,NaN,2515 BT,Zuid-Holland,5c8256cdb04056002cadccfc
4,MAMBOO Sushi,Sushi Restaurant,Raamstraat 15,NL,Den Haag,Nederland,NaN,883,"[Raamstraat 15, 2512BX Den Haag, Nederland]","[{'label': 'display', 'lat': 52.07605214617236...",52.076052,4.309921,NaN,2512BX,Zuid-Holland,592c075c286fda44e172d413
5,SushiPoint Den Haag,Sushi Restaurant,Laan van Meerdervoort 168 B,NL,Den Haag,Nederland,NaN,1519,"[Laan van Meerdervoort 168 B, 2517 BG Den Haag...","[{'label': 'display', 'lat': 52.0817719, 'lng'...",52.081772,4.288170,Zeeheldenkwartier,2517 BG,Zuid-Holland,52308da4498e8f61ae217b7f
6,Sushi Aino,Sushi Restaurant,Laan Van Meerdervoort 322a,NL,Den Haag,Nederland,NaN,1826,"[Laan Van Meerdervoort 322a, Den Haag, Nederland]","[{'label': 'display', 'lat': 52.07887742188163...",52.078877,4.277744,NaN,NaN,Zuid-Holland,525ab1fa498ea60208f9cd62
7,Urban Sushi,Sushi Restaurant,Pletterijkade 15-28,NL,Den Haag,Nederland,NaN,1501,"[Pletterijkade 15-28, 2515 SG Den Haag, Nederl...","[{'label': 'display', 'lat': 52.07438323311686...",52.074383,4.321709,NaN,2515 SG,Zuid-Holland,5112a5bee4b0c57623b89964
8,Sushi den haag centraal,Sushi Restaurant,Den Haag Cs,NL,Den Haag,Nederland,NaN,1619,"[Den Haag Cs, Den Haag, Nederland]","[{'label': 'display', 'lat': 52.070644, 'lng':...",52.070644,4.324361,NaN,NaN,Zuid-Holland,4e131c226284edb6bae5f52c
9,Sushi Tokyo,Sushi Restaurant,Zoutmanstraat 75,NL,Den Haag,Nederland,NaN,1576,"[Zoutmanstraat 75, 2518 GN Den Haag, Nederland]","[{'label': 'display', 'lat': 52.08386092274173...",52.083861,4.293075,NaN,2518 GN,Zuid-Holland,4ecd161e2c5b26aab951d15c


In [31]:
Denhaag_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Denhaag_map)  
    
Denhaag_map

In [34]:
for i in range(0,36):
    x = dataframe_filtered.xs(i)['id']
    venue_id = x
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

    result = requests.get(url).json()
    try:
        print(str(result['response']['venue']['rating'])+",")
    except:
        print("0,")

0,
6.9,
0,
0,
0,
6.4,
0,
0,
0,
0,
0,
0,
0,
0,
8.2,
0,
0,
8.2,
0,
0,
0,
7.3,
0,
0,
6.3,
0,
4.8,
6.4,
7.1,
0,
0,
8.1,
0,
0,
5.9,
7.2,


In [32]:
Ratings = [0,
6.9,
0,
0,
0,
6.4,
0,
0,
0,
0,
0,
0,
0,
0,
8.2,
0,
0,
8.2,
0,
0,
0,
7.3,
0,
0,
6.3,
0,
4.8,
6.4,
7.1,
0,
0,
8.1,
0,
0,
5.9,
7.2,]

In [33]:
rating_frame = pd.DataFrame(Ratings)
rating_frame.head()

,0
0,0.0
1,6.9
2,0.0
3,0.0
4,0.0


In [54]:
conditions = [
    (rating_frame[0] == 0),
    (rating_frame[0] > 0) & (rating_frame[0] <= 5),
    (rating_frame[0] > 5) & (rating_frame[0] <= 6),
    (rating_frame[0] > 6) & (rating_frame[0] <= 7),
    (rating_frame[0] > 7) & (rating_frame[0] <= 8),
    (rating_frame[0] > 8) & (rating_frame[0] <= 9)]
choices = ['0', '1', '2', '3', '4', '5']
choices2 = ['No rating', 'Between 0 and 5', 'Between 5 and 6', 'Between 6 and 7', 'Between 7 and 8', 'Between 8 and 9']
rating_frame['Category'] = np.select(conditions, choices, default='False')
rating_frame['Label'] = np.select(conditions, choices2, default='False')
print(rating_frame)

      0 Category            Label
0   0.0        0        No rating
1   6.9        3  Between 6 and 7
2   0.0        0        No rating
3   0.0        0        No rating
4   0.0        0        No rating
5   6.4        3  Between 6 and 7
6   0.0        0        No rating
7   0.0        0        No rating
8   0.0        0        No rating
9   0.0        0        No rating
10  0.0        0        No rating
11  0.0        0        No rating
12  0.0        0        No rating
13  0.0        0        No rating
14  8.2        5  Between 8 and 9
15  0.0        0        No rating
16  0.0        0        No rating
17  8.2        5  Between 8 and 9
18  0.0        0        No rating
19  0.0        0        No rating
20  0.0        0        No rating
21  7.3        4  Between 7 and 8
22  0.0        0        No rating
23  0.0        0        No rating
24  6.3        3  Between 6 and 7
25  0.0        0        No rating
26  4.8        1  Between 0 and 5
27  6.4        3  Between 6 and 7
28  7.1       

In [55]:
rating_frame = rating_frame.rename(columns={0: 'Rating'})
rating_frame.head(2)

,Rating,Category,Label
0,0.0,0,No rating
1,6.9,3,Between 6 and 7


In [70]:
rating_frame = rating_frame.astype({"Category": int})
rating_frame.dtypes

Rating      float64
Category      int64
Label        object
dtype: object

In [71]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 6
rating_frame_clustering = rating_frame.drop('Label', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(rating_frame_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.YlOrRd(np.linspace(0, 1, len(ys)))
YlOrRd = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, name, Label, category in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name'], rating_frame['Label'], rating_frame['Category']):
    label = folium.Popup(str(name) + ' , Rating: ' + str(Label), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=YlOrRd[category-1],
        fill=True,
        fill_color=YlOrRd[category-1],
        fill_opacity=0.7).add_to(map_clusters)

# makes a map of toronto with all neighborhoods sorted in corresponding boroughs    
map_clusters